# Generalized Data Parser

In [1]:
import os
import numpy as np
from collections import Counter

In [2]:
os.getcwd()

'/home/jango/Documents/EmergenTech'

In [11]:
os.chdir('/home/jango/Documents/EmergenTech/')

In [33]:
def file_content( address ):
    with open( address ) as f:
        x = f.readlines()
    del(x[0])
    xit = iter(x)
    content = []
    for line in xit:
        content.append( line.split() )
    return content
    
def get_sampleIDs( xpress_tbl ):
    x = [line[0] for line in xpress_tbl]
    return list( Counter(x).keys() )

def get_drugs( drug_tbl ):
    x = [line[1] for line in drug_tbl]
    return list( Counter(x).keys() )

# cell line is Lib[<<CATAGORY>>][0].split('/')[-1].split('.')[0]
def Rename( cell_line, sampleIDs, xpress_tbl, drugs_tbl ):
    header=[]
    for smpl in sampleIDs:
        for i in range(len(drugs_tbl)):
            if smpl == drugs_tbl[i][0]:
                header.append( cell_line+'_'+drugs_tbl[i][1]+'_'+smpl.split('_')[1] )
    return header

def get_DiffExp( xpress_tbl, sampleIDs ):
    D={}
    for smpl in sampleIDs:
        tmp=[]
        for line in xpress_tbl:
            if smpl == line[0]:
                tmp.append( line[3] )
        D[smpl] = tmp
    return D

In [35]:
#Get genes symbols (ROW INDICES FOR FINAL MATRIX)
with open('/home/jango/Documents/EmergenTech/L1000genes.txt','r') as f:
    g = f.readlines()

git = iter(g)
Genes=[]
for line in git:
    Genes.append(line.split())
    
Genes = Genes[0]
Genes

['Genes',
 'PSME1',
 'ATF1',
 'RHEB',
 'FOXO3',
 'RHOA',
 'IL1B',
 'ASAH1',
 'RALA',
 'ARHGEF12',
 'SOX2',
 'SERPINE1',
 'HLA-DMA',
 'EGF',
 'APP',
 'NOS3',
 'CSNK1A1',
 'NFATC4',
 'TBP',
 'BRCA1',
 'PSMD4',
 'ETV1',
 'TERT',
 'EED',
 'PTK2B',
 'HSPB1',
 'PIK3C3',
 'CBLB',
 'DFFB',
 'TGFB3',
 'PRKX',
 'CCND1',
 'NFKBIB',
 'GLI2',
 'PIK3CA',
 'RPS6',
 'DFFA',
 'SUZ12',
 'TICAM1',
 'EDN1',
 'SQSTM1',
 'HIF1A',
 'PTGS2',
 'PLA2G4A',
 'MAT2A',
 'EIF4G1',
 'BTK',
 'MYL9',
 'PSMD2',
 'CLTC',
 'ALDOA',
 'STAT5B',
 'ETS1',
 'HSPD1',
 'CEBPA',
 'PSMF1',
 'ATF6',
 'RALB',
 'PPARD',
 'STAT3',
 'DNMT3A',
 'RASA1',
 'PSME2',
 'FGFR2',
 'CALM3',
 'ERO1L',
 'APOE',
 'PIK3R4',
 'CSNK1E',
 'SKP1',
 'PROS1',
 'PSMD9',
 'NFKB2',
 'PAK6',
 'GHR',
 'CDK4',
 'PRKAG2',
 'PTPRC',
 'SPTAN1',
 'COL1A1',
 'CDC25A',
 'TP53',
 'EGR1',
 'MUC1',
 'AURKB',
 'NCK2',
 'AURKA',
 'CDK5R1',
 'FYN',
 'NRAS',
 'BNIP3L',
 'MMP2',
 'CDK7',
 'RAC2',
 'HMOX1',
 'COL4A1',
 'ICAM1',
 'C5',
 'NFKBIA',
 'NFKBIE',
 'CCNH',
 'HMGA2',

In [36]:
path = '/home/jango/Documents/EmergenTech/Repurpose_parsed/'
os.chdir(path)
Classes = os.listdir()

In [37]:
Classes

['Anti-HIV_Agents',
 'Immunologic_Factors',
 'Membrane_Transport_Modulator',
 'Anti-Inflammatory_Agents',
 'Anti-Bacterial_Agents',
 'Enzyme_Inhibitor']

In [38]:
label_dict = {lb:idx for idx,lb in enumerate(Classes)}

In [39]:
label_dict

{'Anti-Bacterial_Agents': 4,
 'Anti-HIV_Agents': 0,
 'Anti-Inflammatory_Agents': 3,
 'Enzyme_Inhibitor': 5,
 'Immunologic_Factors': 1,
 'Membrane_Transport_Modulator': 2}

In [44]:
Lib = Counter()
for category in Classes:
    Lib[str(category)] = []

for category in Classes:
    os.chdir(path+category+'/')
    files = os.listdir()
    x = []
    for file in files:
        if file != 'ZZactions_LIB.xls':
            x.append(path+category+'/'+file)
    Lib[str(category)] = x

In [45]:
A=list()
A.append( Genes )

Labels=list()
for category in Classes:
    
    # The drug table to the specific category & drug names
    drug_table = file_content( path+category+'/'+'ZZactions_LIB.xls' )
    
    # Different cell line differential-expression data
    for xprs in Lib[category][0:3]:
        
        # Cell Type expression data
        CTx = file_content( xprs )
        
        # Which Cell Line
        cell = xprs.split('/')[-1].split('.')[0]
        
        # Get Sample ID's
        sampleIDs = get_sampleIDs( CTx )
        
        # Get expression values for each sample ID <<THIS IS A DICTIONARY>>
        Diffx = get_DiffExp( CTx, sampleIDs )
        
        # Get the renamed sample ID's (HEADER FOR FINAL MATRIX)
        header = Rename( cell, sampleIDs, CTx, drug_table )
        
        # Append to Labels matrix
        for elem in header:
            Labels.append( (elem, label_dict[category]) )
        
        # Append to the final data matrix
        for k,name in zip(Diffx.keys(),header):
            A.append( [name]+Diffx[k] )
            


In [46]:
drug_table[0:10]

[['LINCSCP_4232', 'Indomethacin', '10uM', 'A549', '24h'],
 ['LINCSCP_44740', 'Indomethacin', '10uM', 'PC3', '24h'],
 ['LINCSCP_44401', 'Enalapril', '10uM', 'PC3', '24h'],
 ['LINCSCP_6886', 'Enalapril', '10uM', 'A549', '24h'],
 ['LINCSCP_47997', 'Gabexate', '10uM', 'PC3', '6h'],
 ['LINCSCP_5756', 'Gabexate', '10uM', 'A549', '24h'],
 ['LINCSCP_5878', 'Gabexate', '10uM', 'A549', '6h'],
 ['LINCSCP_33387', 'Ubenimex', '10uM', 'MCF7', '24h'],
 ['LINCSCP_7487', 'Ubenimex', '10uM', 'A549', '6h'],
 ['LINCSCP_6888', 'Quinidine', '10uM', 'A549', '24h']]

In [47]:
Labels[0:10]

[('A549_Ubenimex_33387', 0),
 ('A549_Lamivudine_34563', 0),
 ('A549_Tenofovir_32462', 0),
 ('A549_Tenofovir_32659', 0),
 ('A549_Indinavir_31762', 0),
 ('A549_Zidovudine_34306', 0),
 ('A549_Zalcitabine_34734', 0),
 ('A549_Stavudine_31777', 0),
 ('A549_Stavudine_34737', 0),
 ('A549_Ubenimex_7487', 0)]

In [48]:
len(Labels)

2346

In [49]:
A[0:10]

[['Genes',
  'PSME1',
  'ATF1',
  'RHEB',
  'FOXO3',
  'RHOA',
  'IL1B',
  'ASAH1',
  'RALA',
  'ARHGEF12',
  'SOX2',
  'SERPINE1',
  'HLA-DMA',
  'EGF',
  'APP',
  'NOS3',
  'CSNK1A1',
  'NFATC4',
  'TBP',
  'BRCA1',
  'PSMD4',
  'ETV1',
  'TERT',
  'EED',
  'PTK2B',
  'HSPB1',
  'PIK3C3',
  'CBLB',
  'DFFB',
  'TGFB3',
  'PRKX',
  'CCND1',
  'NFKBIB',
  'GLI2',
  'PIK3CA',
  'RPS6',
  'DFFA',
  'SUZ12',
  'TICAM1',
  'EDN1',
  'SQSTM1',
  'HIF1A',
  'PTGS2',
  'PLA2G4A',
  'MAT2A',
  'EIF4G1',
  'BTK',
  'MYL9',
  'PSMD2',
  'CLTC',
  'ALDOA',
  'STAT5B',
  'ETS1',
  'HSPD1',
  'CEBPA',
  'PSMF1',
  'ATF6',
  'RALB',
  'PPARD',
  'STAT3',
  'DNMT3A',
  'RASA1',
  'PSME2',
  'FGFR2',
  'CALM3',
  'ERO1L',
  'APOE',
  'PIK3R4',
  'CSNK1E',
  'SKP1',
  'PROS1',
  'PSMD9',
  'NFKB2',
  'PAK6',
  'GHR',
  'CDK4',
  'PRKAG2',
  'PTPRC',
  'SPTAN1',
  'COL1A1',
  'CDC25A',
  'TP53',
  'EGR1',
  'MUC1',
  'AURKB',
  'NCK2',
  'AURKA',
  'CDK5R1',
  'FYN',
  'NRAS',
  'BNIP3L',
  'MMP2',
  'C

In [50]:
len(A)

2347

# Save The Data

In [63]:
fin = np.array( list( zip(*A) ) )

In [75]:
import pandas as pd 
df = pd.DataFrame(data=fin[1:,1:], index=fin[1:,0], columns=fin[0,1:])
df.to_csv("/home/jango/Documents/EmergenTech/L1000dataset.csv")

In [74]:
df.head()

,A549_Ubenimex_33387,A549_Lamivudine_34563,A549_Tenofovir_32462,A549_Tenofovir_32659,A549_Indinavir_31762,A549_Zidovudine_34306,A549_Zalcitabine_34734,A549_Stavudine_31777,A549_Stavudine_34737,A549_Ubenimex_7487,...,PC3_252916-29-3_46852,PC3_Cerivastatin_46118,PC3_Entacapone_47341,PC3_Cilastatin_46124,PC3_Triacsin_45231,PC3_Radicicol_43903,PC3_Radicicol_43935,PC3_Radicicol_67973,PC3_Radicicol_67994,PC3_Atorvastatin_45069
PSME1,-0.71402,-0.261,-0.233967,-0.20424,0.3866,-0.105767,0.35075,0.38545,-0.19575,-0.854333,...,0.62534,0.229225,0.04358,0.254875,-0.04875,0.499217,0.965312,-1.99568,-0.708744,-0.282367
ATF1,-0.83916,-0.865733,-0.229533,-1.34262,-3.33905,0.1477,-1.401,-2.8018,0.39535,-1.00917,...,0.30902,-0.31035,1.64042,0.77185,0.659525,0.213408,-0.612675,-0.219567,-1.70359,-0.49235
RHEB,-0.26298,-0.962867,-0.514633,-0.64016,0.437,0.263433,1.1425,0.41765,-0.2602,-0.2148,...,0.61842,0.30425,0.17078,0.11375,0.31355,0.608542,-0.698775,0.468811,1.27274,0.0534
FOXO3,-0.23738,-0.539067,0.389833,-0.06742,0.0501,0.454067,0.7028,-0.4148,0.8945,-1.1378,...,0.9594,3.09683,0.61976,-0.364675,-0.261175,0.0794917,0.70945,-0.376867,1.46091,1.62822
RHOA,-0.06352,0.0495333,0.288767,1.4081,0.17155,-0.0123,0.7676,0.69215,0.8744,-0.856633,...,0.60308,4.55755,-0.9088,0.236425,0.17915,-1.18278,0.391937,-1.54341,-0.176822,5.33858


In [76]:
Probes = np.array( Labels )

In [77]:
Probes[0:10]

array([['A549_Ubenimex_33387', '0'],
       ['A549_Lamivudine_34563', '0'],
       ['A549_Tenofovir_32462', '0'],
       ['A549_Tenofovir_32659', '0'],
       ['A549_Indinavir_31762', '0'],
       ['A549_Zidovudine_34306', '0'],
       ['A549_Zalcitabine_34734', '0'],
       ['A549_Stavudine_31777', '0'],
       ['A549_Stavudine_34737', '0'],
       ['A549_Ubenimex_7487', '0']], 
      dtype='<U64')

In [84]:
lb = pd.DataFrame(Probes, columns=['SampleID', 'Class'])

In [85]:
lb.head()

,SampleID,Class
0,A549_Ubenimex_33387,0
1,A549_Lamivudine_34563,0
2,A549_Tenofovir_32462,0
3,A549_Tenofovir_32659,0
4,A549_Indinavir_31762,0


In [86]:
lb.to_csv("/home/jango/Documents/EmergenTech/Labels.csv", index=False)